## TreResNet on CIFAR
* Use ResNet50, Densenet 121
* best 4 out of 8 networks 
* Compare accuracy for 1 to 4 streams
* Use standard autgmentation, no test time

In [1]:
import os
import sys
import random
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from IPython.display import display, HTML

import scipy.io

%reload_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler

import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet50, densenet121
from torchvision import datasets, models, transforms


%reload_ext autoreload
%autoreload 2
%matplotlib inline

PATH = Path('/home/rene/data/cifar')

# Add the src directory for functions
src_dir = Path.cwd().parent.parent / 'src'
print(src_dir)
sys.path.append(str(src_dir))

# import my functions:
from utils import make_cfiar10
from models import*
from functions import*

print(torch.cuda.device_count())
print(torch.cuda.is_available())
print(torch.cuda.current_device())

/home/rene/code/idc/src
1
True
0


## Get dataset, convert to keras format, make sample

In [5]:
# ! wget https://pjreddie.com/media/files/cifar.tgz -P /home/rene/data/
# ! tar xzf /home/rene/data/cifar.tgz -C /home/rene/data
# ! mv /home/rene/data/cifar /home/rene/data/cifar_raw

# PATH_IN = Path('/home/rene/data/cifar_raw')
# PATH_OUT = PATH
# make_cfiar10(PATH_IN, PATH_OUT)

URL transformed to HTTPS due to an HSTS policy
--2018-06-18 19:31:33--  https://pjreddie.com/media/files/cifar.tgz
Resolving pjreddie.com (pjreddie.com)... 128.208.3.39
Connecting to pjreddie.com (pjreddie.com)|128.208.3.39|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168584360 (161M) [application/octet-stream]
Saving to: ‘/home/rene/data/cifar.tgz’

cifar.tgz           100%[===================>] 160.77M  37.7MB/s    in 4.7s    

2018-06-18 19:31:39 (34.2 MB/s) - ‘/home/rene/data/cifar.tgz’ saved [168584360/168584360]



## Train 8 Individial ResNet 50

In [ ]:
PATH = Path('/home/rene/data/cifar')
save_path = PATH / 'models'
save_path.mkdir(parents=True, exist_ok=True)

num_workers = 6
batch_size=80
epochs = 40

models = {}
performance = {}
for i in range(8):
    models['resnet50_'+str(i)] = resnet50
    
dataloaders, dataset_sizes = make_batch_gen(str(PATH), batch_size, num_workers, 
                                            valid_name='valid', size=197)

for model_name, model_arch in models.items():
    model = model_arch(pretrained=False)
    model.fc = nn.Linear(model.fc.in_features, 10)
    model = model.cuda()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.005)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=int(epochs/3), gamma=0.1)

    best_acc, model = train_model(model, criterion, optimizer, exp_lr_scheduler,
                                     epochs, dataloaders, dataset_sizes)
    torch.save(model.state_dict(), str(save_path / model_name))

Epoch 0/39
----------


In [2]:
'''Pre-activation ResNet in PyTorch.
Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Identity Mappings in Deep Residual Networks. arXiv:1603.05027
'''
import torch
import torch.nn as nn
import torch.nn.functional as F


class PreActBlock(nn.Module):
    '''Pre-activation version of the BasicBlock.'''
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(PreActBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)

        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = F.relu(self.bn1(x))
        shortcut = self.shortcut(out) if hasattr(self, 'shortcut') else x
        out = self.conv1(out)
        out = self.conv2(F.relu(self.bn2(out)))
        out += shortcut
        return out


class PreActBottleneck(nn.Module):
    '''Pre-activation version of the original Bottleneck module.'''
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(PreActBottleneck, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)

        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = F.relu(self.bn1(x))
        shortcut = self.shortcut(out) if hasattr(self, 'shortcut') else x
        out = self.conv1(out)
        out = self.conv2(F.relu(self.bn2(out)))
        out = self.conv3(F.relu(self.bn3(out)))
        out += shortcut
        return out


class PreActResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(PreActResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def PreActResNet18():
    return PreActResNet(PreActBlock, [2,2,2,2])

def PreActResNet34():
    return PreActResNet(PreActBlock, [3,4,6,3])

def PreActResNet50():
    return PreActResNet(PreActBottleneck, [3,4,6,3])

def PreActResNet101():
    return PreActResNet(PreActBottleneck, [3,4,23,3])

def PreActResNet152():
    return PreActResNet(PreActBottleneck, [3,8,36,3])


def test():
    net = PreActResNet18()
    y = net((torch.randn(1,3,32,32)))
    print(y.size())


In [5]:
best_acc = 0  # best test accuracy
epochs = 50



transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='/home/rene/data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=80, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='/home/rene/data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=80, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

net = PreActResNet50()
device = 'cuda'

net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=.005, momentum=0.9, weight_decay=5e-4)
scheduler = lr_scheduler.StepLR(optimizer, step_size=int(epochs/3), gamma=0.1)

def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        scheduler.step()
        inputs, targets = inputs.to(device), targets.to(device)
#         inputs = Variable(inputs.cuda())
#         targets = Variable(targets.cuda())
        
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    print(100.*correct/total, correct, total)

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    print('test:', 100.*correct/total, correct, total)

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        torch.save(state, str(save_path / 'resnet50_other'))
        best_acc = acc

Files already downloaded and verified
Files already downloaded and verified


In [6]:
for epoch in range(epochs):
    train(epoch)
    test(epoch)


Epoch: 0
18.638 9319 50000
test: 18.67 1867 10000
Saving..


NameError: name 'save_path' is not defined

## Train Parallel

In [ ]:
PATH = Path('/home/rene/data/cifar')
save_path = PATH / 'models'
save_path.mkdir(parents=True, exist_ok=True)

num_workers = 6
batch_size=80
epochs = 20

models = {}
performance = {}
for i in range(8):
    models['resnet50_'+str(i)] = resnet50
    
dataloaders, dataset_sizes = make_batch_gen(str(PATH), batch_size, num_workers, 
                                            valid_name='valid', size=197)

for model_name, model_arch in models.items():
    model = model_arch(pretrained=False)
    model.fc = nn.Linear(model.fc.in_features, 10)
    model = model.cuda()
    model = torch.nn.DataParallel(model, device_ids=[0, 1])

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.005)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.1)

    best_acc, model = train_model_multi(model, criterion, optimizer, exp_lr_scheduler,
                                     epochs, dataloaders, dataset_sizes)
    torch.save(model.state_dict(), str(save_path / model_name))



## Dense Net 121

In [ ]:
PATH = Path('/home/rene/data/cifar/sample')
num_workers = 6
batch_size=32
epochs = 15

models = {}
for i in range(8):
    models['densenet121_'+str(i)] = densenet121
    
dataloaders, dataset_sizes = make_batch_gen(str(PATH), batch_size, num_workers, valid_name='valid', size=224)

for model_name, model_arch in models.items():
    model = model_arch(pretrained=False)
    model.fc = nn.Linear(model.fc.in_features, 10)
    model = model.cuda()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.003)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

    best_acc, model = train_model(model, criterion, optimizer, exp_lr_scheduler,
                                     epochs, dataloaders, dataset_sizes)
    torch.save(model.state_dict(), str(PATH /'models' / model_name))